In [78]:
import arxiv

# Construct the default API client.
client = arxiv.Client()

search = arxiv.Search(
    query = "Attention is all you need",
    max_results = 10,
    sort_by=arxiv.SortCriterion.SubmittedDate, 
    sort_order=arxiv.SortOrder.Descending
)

results = client.results(search)
# `results` is a generator; you can iterate over its elements one by one...
for r in client.results(search):
    print(r.download_pdf())
    print()
    break

./2404.15626v1.An_Electromagnetism_Inspired_Method_for_Estimating_In_Grasp_Torque_from_Visuotactile_Sensors.pdf



## Инициализация модели
Теперь инициализируем модель GigaChat.

In [150]:
with open("credentials.txt", 'r', encoding='utf-8') as file:
    credentials = file.read()

In [151]:
from langchain.chat_models.gigachat import GigaChat

llm = GigaChat(credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False)

In [176]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("assets/arxiv_paper.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

Total documents: 48


После нарезки мы получили 91 документ частями книги.

## Создание базы данных эмбеддингов

Эмбеддинг это векторное представление текста, которое может быть использовано для определения смысловой близости текстов. Векторная база данных хранит тексты и соответствующие им эмбеддинги, а также умеет выполнять поиск по ним. Для работы с базой данных мы создаем объект GigaChatEmbeddings и передаем его в базу данных Chroma.

> Обратите внимание, что сервис для вычисления эмбеддингов может тарифицироваться отдельно от стоимости модели GigaChat.

In [177]:
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

embeddings = GigaChatEmbeddings(
    credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False
)

db = Chroma.from_documents(
    documents,
    embeddings,
    client_settings=Settings(anonymized_telemetry=False),
)

## Поиск по базе данных

Теперь можно обратиться к базе данных и попросить найти документы, которые с наибольшей вероятностью содержат ответ на наш вопрос.

По-умолчанию база данных возвращает 4 наиболее релевантных документа. Этот параметр можно изменить в зависимости от решаемой задачи и типа документов.

Видно, что первый же документ содержит внутри себя часть книги с ответом на наш вопрос.

In [178]:
docs = db.similarity_search(question, k=4)
len(docs)

4

In [179]:
print(f"... {str(docs[0])[620:800]} ...")

... ение (fork) открытой библиотеки LangСhain на Python. Её главная цель — облегчить жизнь разработчику. Библиотека состоит из большого количества различных компонентов, которые позвол ...


## QnA цепочка

Теперь мы создадим цепочку QnA, которая специально предназначена для ответов на вопросы по документам. В качестве аргументов есть передается языковая модель и ретривер (база данных).

In [180]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever(), return_source_documents=True)

Наконец можно задать вопрос нашей цепочке и получить правильный ответ!

In [181]:
qa_chain({"query": question})

{'query': 'Какой Loss использует Yolov8?',
 'result': 'Я не знаю ответа на этот вопрос.',
 'source_documents': [Document(page_content='Какой плащ был у Понтия Пилата? Отвечает GigaChat\nСредний\n10 мин\n6.4K\nБлог компании SberDevices\nМашинное обучение\n*\nИскусственный интеллект\nNatural Language Processing\n*\nОбзор\n\nВсем привет! На связи лид разработки SDK GigaChat’a — Константин Крестников. В этой статье я расскажу о том, что такое GigaChain и как в целом SDK позволяет упростить жизнь разработчика LLM, например, научить LLM давать ответы на вопросы по вашим документам или работать в режиме автономного агента. Также поговорим про решения, которые практически невозможно сделать без использования SDK.\n\nGigaСhain — это ответвление (fork) открытой библиотеки LangСhain на Python. Её главная цель — облегчить жизнь разработчику. Библиотека состоит из большого количества различных компонентов, которые позволяют работать с промптами, объединять вызовы к большим языковым моделям в цепочк

Несколько дополнительных вопросов для проверки работоспособности:

In [89]:
qa_chain({"query": "Расскажи про тимлида Сбера"})

{'query': 'Расскажи про тимлида Сбера',
 'result': 'Извините, но у меня нет информации о конкретном тимлиде Сбера. Могу рассказать вам о том, что такое тимлид и какие у него обязанности. Тимлид - это лидер команды, который отвечает за управление командой, ее развитие и достижение поставленных целей. Он координирует работу членов команды, помогает им решать возникающие проблемы, поддерживает мотивацию и вовлеченность. Кроме того, тимлид также участвует в принятии решений, связанных с проектами, и помогает команде в достижении общих целей.'}

In [182]:
qa_chain({"query": "Основные особенности GigaChain"})

{'query': 'Основные особенности GigaChain',
 'result': 'GigaChain обладает следующими особенностями:\n\n1. Обратная совместимость с LangChain: GigaChain может использоваться вместе с другими LLM в различных комбинациях.\n\n2. Большое количество инструментов и примеров: GigaChain содержит большое количество инструментов и примеров, которые можно использовать в качестве основы своего проекта или просто изучать для расширения кругозора в области возможных кейсов применения LLM.\n\n3. Реализация агентов: GigaChain позволяет создавать агентов, которые могут взаимодействовать с внешней средой и корректировать свое поведение в зависимости от результатов взаимодействия.\n\n4. Загрузка документов: GigaChain позволяет загружать документы пользователя в память и разрезать их на части для дальнейшего анализа.\n\n5. Работа с эмбеддингами: Каждая часть документа преобразуется в эмбеддинг - числовой вектор, который представляет собой внутреннее представление текста моделью.\n\n6. Работа с базами данн

In [125]:
import arxiv
import os
import json

def download_from_arxiv(key_word, max_results=10, destination_path="assets/arxiv/", saved_list_path="assets/arxiv/papers.json"):
    client = arxiv.Client()

    search = arxiv.Search(
        query = str(key_word),
        max_results = max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate, 
        sort_order=arxiv.SortOrder.Descending
    )
    # Проверка существования файла с сохраненным списком скачанных arXiv ID
    if os.path.exists(saved_list_path):
        with open(saved_list_path, "r") as f:
            downloaded_arxiv_ids = json.load(f)
    else:
        downloaded_arxiv_ids = []
    
    for result in client.results(search):
        # Проверка наличия статьи в списке уже скачанных
        if result.entry_id.split("/")[-1] in downloaded_arxiv_ids:
            print(f"Статья {result.entry_id} уже скачана и пропущена.")
            continue
        
        # Скачивание PDF-файла статьи
        pdf_path = result.download_pdf(dirpath=destination_path)
        if pdf_path:
            print(f"Статья {result.entry_id} успешно скачана и сохранена в {pdf_path}")
            # Добавление arXiv ID в список скачанных
            downloaded_arxiv_ids.append(result.entry_id.split("/")[-1])
    # Сохранение списка скачанных arXiv ID в файл JSON
    with open(saved_list_path, "w") as f:
        json.dump(downloaded_arxiv_ids, f)

In [127]:
key_words = ["NLP", "RAG", "ChatBot", "LLM", "Speech Recognition", "LangChain", "LLM Agents"]

for key_word in key_words:
    # Пример использования функции
    download_from_arxiv(key_word, max_results=10)

Статья http://arxiv.org/abs/2404.15238v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.15104v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14977v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14963v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14943v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.15382v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14809v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14740v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14695v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14631v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.15488v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14043v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.13948v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.13892v2 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.13781v1 уже скачана и пропущена.
Статья htt

In [118]:
def get_pdf_files(directory):
    pdf_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".pdf"):
                pdf_files.append(os.path.join(root, file))
    return pdf_files

In [119]:
# Пример использования функции
directory = "assets/arxiv/"
pdf_files = get_pdf_files(directory)
for pdf_file in pdf_files:
    print(pdf_file)

assets/arxiv/2404.15604v1.Hybrid_LLM_Rule_based_Approaches_to_Business_Insights_Generation_from_Structured_Data.pdf
assets/arxiv/2404.15588v1.Minimal_Evidence_Group_Identification_for_Claim_Verification.pdf
assets/arxiv/2404.15578v1.Can_Foundational_Large_Language_Models_Assist_with_Conducting_Pharmaceuticals_Manufacturing_Investigations_.pdf
assets/arxiv/2404.15564v1.Guided_AbsoluteGrad__Magnitude_of_Gradients_Matters_to_Explanation_s_Localization_and_Saliency.pdf
assets/arxiv/2404.15592v1.ImplicitAVE__An_Open_Source_Dataset_and_Multimodal_LLMs_Benchmark_for_Implicit_Attribute_Value_Extraction.pdf
assets/arxiv/2404.13892v2.Retrieval_Augmented_Audio_Deepfake_Detection.pdf
assets/arxiv/2404.14631v1.Learning_Word_Embedding_with_Better_Distance_Weighting_and_Window_Size_Scheduling.pdf
assets/arxiv/2404.15552v1.Cross_Temporal_Spectrogram_Autoencoder__CTSAE___Unsupervised_Dimensionality_Reduction_for_Clustering_Gravitational_Wave_Glitches.pdf
assets/arxiv/2404.14901v1.Beyond_Code_Generation

In [120]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader


def summarize(path_to_pdf_file, chunk_size=5000, chunk_overlap=1000): 
    loader = PyMuPDFLoader(path_to_pdf_file)
    docs = loader.load()
    split_docs = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap).split_documents(
        docs
    )
    
    giga = GigaChat(credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False)
    chain = load_summarize_chain(giga, chain_type="map_reduce")
    res = chain.run(split_docs)
    return res

In [131]:
import sqlite3

class ArticlesDB:
    def __init__(self, db_path='articles.db'):
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self.create_table()

    def create_table(self):
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS articles
                            (id INTEGER PRIMARY KEY,
                            title TEXT NOT NULL,
                            summary TEXT)''')
        self.conn.commit()

    def add_article(self, title, summary):
        self.cursor.execute('INSERT INTO articles (title, summary) VALUES (?, ?)', (title, summary))
        self.conn.commit()

    def delete_article(self, title):
        self.cursor.execute('DELETE FROM articles WHERE title = ?', (title,))
        self.conn.commit()

    def get_article_summary(self, title):
        self.cursor.execute('SELECT summary FROM articles WHERE title = ?', (title,))
        result = self.cursor.fetchone()
        if result:
            return result[0]
        else:
            return None

    def get_all_articles(self):
        self.cursor.execute('SELECT title, summary FROM articles')
        return self.cursor.fetchall()

    def close(self):
        self.conn.close()


In [136]:
articles_db = ArticlesDB(db_path="articles.db")

In [ ]:
from tqdm import tqdm

for filename in tqdm(pdf_files, desc="Processing PDF files"):
    article_title = filename.split("/")[-1][:-4]
    article_summary = articles_db.get_article_summary(article_title)
    if article_summary is None:
        result = summarize(filename,
                       chunk_size=5000,
                       chunk_overlap=1000)
        articles_db.add_article(article_title, result)
        print(f"Article \"{article_title}\" saved to database")
print(articles_db.get_all_articles())

In [143]:
for name, summary in articles_db.get_all_articles():
    print(name)
    print(summary)
    print()

2404.15604v1.Hybrid_LLM_Rule_based_Approaches_to_Business_Insights_Generation_from_Structured_Data
The paper discusses the integration of rule-based systems and Large Language Models (LLMs) to generate actionable business insights from structured data. The hybrid approach combines the robustness of rule-based systems with the adaptive power of LLMs, addressing the complexities of data extraction in the field of business intelligence. The hybrid approach combines interpretable AI techniques, such as LIME, rule-based systems, and supervised document classification, to extract actionable insights from business data. LLM plays a vital role in this process by enhancing the understanding and generation of natural language-generated insights. The hybrid approach can uncover personalized and nuanced user interests, needs, and goals from user journeys and user activities on the platform. However, when implementing this approach, it is essential to consider data quality, domain knowledge, and sc

In [1]:
from langchain.chat_models.gigachat import GigaChat

with open("credentials.txt", 'r', encoding='utf-8') as file:
    credentials = file.read()
    
llm = GigaChat(credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False)

In [2]:
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain.vectorstores.faiss import FAISS
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

embeddings = GigaChatEmbeddings(
    credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False
)

faiss_db = FAISS.from_documents(documents, embedding=embeddings)

NameError: name 'documents' is not defined

In [3]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

loader = PyPDFDirectoryLoader("assets/arxiv/")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

KeyboardInterrupt: 

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [160]:
qa_chain({"query": "Расскажи про Retrieval-Augmented Audio Deepfake Detection"})

{'query': 'Расскажи про Retrieval-Augmented Audio Deepfake Detection',
 'result': 'Извините, но я не могу найти информацию о "Retrieval-Augmented Audio Deepfake Detection". Можете ли вы уточнить, что именно вы хотите узнать о нем?'}

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document

In [5]:
from typing import Any, Coroutine, List


class HuggingFaceE5Embeddings(HuggingFaceEmbeddings):
    def embed_query(self, text: str) -> List[float]:
        text = f"query: {text}"
        return super().embed_query(text)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        texts = [f"passage: {text}" for text in texts]
        return super().embed_documents(texts)

    async def aembed_query(self, text: str) -> Coroutine[Any, Any, List[float]]:
        text = f"query: {text}"
        return await super().aembed_query(text)

    async def aembed_documents(
        self, texts: List[str]
    ) -> Coroutine[Any, Any, List[List[float]]]:
        texts = [f"passage: {text}" for text in texts]
        return await super().aembed_documents(texts)

In [6]:
embedding = HuggingFaceE5Embeddings(model_name="intfloat/multilingual-e5-base")

In [7]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.vectorstores.faiss import FAISS

loader = PyPDFDirectoryLoader("assets/arxiv/")
documents = loader.load()

faiss_db = FAISS.from_documents(documents, embedding=embedding)

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 66 0 (offset 0)
Ignoring wrong pointing object 68 0 (offset 0)
Ignoring wrong pointing object 80 0 (offset 0)
Ignoring wrong pointing object 85 0 (offset 0)
Ignoring wrong pointing object 232 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 44 0 (offset 0)
Ignoring wrong p

In [8]:
import string

def tokenize(s: str) -> list[str]:
    """Очень простая функция разбития предложения на слова"""
    return s.lower().translate(str.maketrans("", "", string.punctuation)).split(" ")

embedding_retriever = faiss_db.as_retriever(search_kwargs={"k": 2})
bm25_retriever = BM25Retriever.from_documents(
    documents=documents,
    preprocess_func=tokenize,
    k=3,
)
ensemble_retriever = EnsembleRetriever(
    retrievers=[embedding_retriever, bm25_retriever],
    weights=[0.4, 0.6],
)

In [9]:
from langchain.chains import RetrievalQA
from langchain.llms.gigachat import GigaChat

llm = GigaChat(credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=ensemble_retriever,
    return_source_documents=True,
)
qa.invoke({"query": "Что такое RAG?"})

{'query': 'Что такое вода?',
 'result': 'Вода - это прозрачная бесцветная жидкость без запаха, которая составляет большую часть поверхности Земли и является необходимым условием для жизни большинства живых организмов. Она играет важную роль в различных процессах, таких как поддержание температуры, обеспечение питания и поддержание экосистемы. Вода также используется в различных отраслях промышленности, включая производство продуктов питания, текстиля, химических веществ и энергетики.',
 'source_documents': [Document(page_content='Hybrid\nLLM/Rule-based\nApproaches\nto\nBusiness\nInsights\nGeneration\nfrom\nStructured\nData\nAliaksei\nVertsel\nand\nMikhail\nRumiantsau\nNarrative\nBI\n(\nteam@narrative.bi\n)\nAbstract\nIn\nthe\nfield\nof\nbusiness\ndata\nanalysis,\nthe\nability\nto\nextract\nactionable\ninsights\nfrom\nvast\nand\nvaried\ndatasets\nis\nessential\nfor\ninformed\ndecision-making\nand\nmaintaining\na\ncompetitive\nedge.\nTraditional\nrule-based\nsystems,\nwhile\nreliable,\no

In [10]:
qa.invoke({"query": "Что такое RAG?"})

{'query': 'Что такое RAG?',
 'result': 'RAG - это аббревиатура от "Retrieval-Augmented Generation", что в переводе означает "Усиление за счет поиска". Это метод обработки естественного языка, который сочетает в себе преимущества поиска и генерации. Он используется для улучшения качества ответов на вопросы и решения других задач в области обработки естественного языка.',
 'source_documents': [Document(page_content='Typos that Broke the RAG’s Back: Genetic Attack on RAG Pipeline\nby Simulating Documents in the Wild via Low-level Perturbations\nSukmin Cho Soyeong Jeong Jeongyeon Seo Taeho Hwang Jong C. Park*\nSchool of Computing\nKorea Advanced Institute of Science and Technology\n{nelllpic,starsuzi,yena.seo,doubleyyh,jongpark}@kaist.ac.kr\nAbstract\nThe robustness of recent Large Language Mod-\nels (LLMs) has become increasingly crucial\nas their applicability expands across various\ndomains and real-world applications. Retrieval-\nAugmented Generation (RAG) is a promising\nsolution for 

In [11]:
qa.invoke({"query": "Кратко расскажи про особенности RAD"})

{'query': 'Кратко расскажи про особенности RAD',
 'result': 'RAD (Retrieval-Augmented Detection) - это метод, который используется для улучшения процесса обнаружения. Он состоит из трех основных этапов: построение базы знаний, поиск знаний и получение результатов. В процессе построения базы знаний, аудиоданные разбиваются на более мелкие фрагменты, которые затем кодируются в латентные представления. Затем эти представления преобразуются в векторные представления, что позволяет проводить эффективный поиск. В процессе поиска знаний, векторные представления используются для поиска наиболее похожих аудиофрагментов. Полученные результаты затем используются для принятия решения о подлинности аудиоданных.',
 'source_documents': [Document(page_content='ICMR ’24, June 10–14, 2024, Phuket, Thailand Zuheng Kang et al.\nFigure 3: The overview of the RAG and RAD pipeline. Triangular edge rectangles represent vectors for retrieval databases.\nIn RAG, long rectangles represent document chunks. In RAD

In [12]:
qa.invoke({"query": "Представь несколько идей по использованию RAG системы"})

{'query': 'Представь несколько идей по использованию RAG системы',
 'result': 'RAG система может быть использована для улучшения качества и точности генерации текста в различных областях, таких как вопросы и ответы, факт-чекинг и диалоговое взаимодействие. Она может быть полезна для создания более релевантных и точных ответов на вопросы, а также для проверки фактов и обеспечения достоверности информации. Кроме того, RAG система может быть использована для улучшения персонализации и адаптации контента к потребностям и интересам пользователей.',
 'source_documents': [Document(page_content='Typos that Broke the RAG’s Back: Genetic Attack on RAG Pipeline\nby Simulating Documents in the Wild via Low-level Perturbations\nSukmin Cho Soyeong Jeong Jeongyeon Seo Taeho Hwang Jong C. Park*\nSchool of Computing\nKorea Advanced Institute of Science and Technology\n{nelllpic,starsuzi,yena.seo,doubleyyh,jongpark}@kaist.ac.kr\nAbstract\nThe robustness of recent Large Language Mod-\nels (LLMs) has beco